In [8]:
from slovnet import Syntax
from slovnet import Morph
from slovnet import NER

import slovnet
from navec import Navec
from razdel import sentenize, tokenize

import json
import os

In [2]:
# import rutokenizer
# import rupostagger
# import rulemma

# lemmatizer = rulemma.Lemmatizer()
# lemmatizer.load()
# tokenizer = rutokenizer.Tokenizer()
# tokenizer.load()
# tagger = rupostagger.RuPosTagger()
# tagger.load()

In [3]:
stopwords = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но',
 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня',
 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже',
 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть',
 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам',
 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они',
 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя',
 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего',
 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто',
 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь',
 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас',
 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец',
 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот',
 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много',
 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой',
 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им',
 'более', 'всегда', 'конечно', 'всю', 'между']

In [12]:
#load text

def create_parses(file_name, parsers = [None, None, None]):
    if any(parsers) is None:
        return None
    syntax, morph, ner = parsers
    #загрузка текста
    with open(file_name, 'r') as fin:
        data = json.load(fin)
    text = data['textInFile']
    #предобработка для "лексического анализа"
    ner_parse = ner(text)
    
    # tokens = tokenizer.tokenize(text)
    # tags = tagger.tag(tokens)
    # lemmas = lemmatizer.lemmatize(tags)
    lemmatized = ''
    # for tok in lemmas:
    #     if not tok[2] in stopwords:
    #         lemmatized += tok[2] + ' '
    #разчиение на предложения
    document = [sent.text for sent in sentenize(text)]
    #разбиение на токены
    tokens = []
    for sent in document:
        tokenised_sentence = [_.text for _ in tokenize(sent)]
        tokens.append(tokenised_sentence)

        for tok in tokenised_sentence:
            if not tok in stopwords:
                lemmatized += tok + ' '
    #парсинг
    syntax_parse = []
    morph_parse = []
    
    for sent in tokens:
        syntax_parse.append(next(syntax.map([sent])).tokens)
        morph_parse.append(next(morph.map([sent])).tokens)

    return syntax_parse, morph_parse, ner_parse, lemmatized

In [13]:
#saving parses

def save_parses(file_name, syntax_parse, morph_parse, ner_parse, lemmatized):

    syntx_file_name = file_name + '.' + 'syntx'
    morph_file_name = file_name + '.' + 'morph'
    ner_file_name = file_name + '.' + 'ner'
    lem_file_name = file_name + '.' + 'lem'

    syntx_array = []
    morph_array = []

    for sent in syntax_parse:
        syntax_subarray = []
        for token in sent:
            syntax_subarray.append(token.as_json)
        syntx_array.append(syntax_subarray)
        
    for sent in morph_parse:
        morph_subarray = []
        for token in sent:
            morph_subarray.append(token.as_json)
        morph_array.append(morph_subarray)
        

    with open(syntx_file_name, 'w') as syn_out:
        json.dump(syntx_array, syn_out)
    with open(morph_file_name, 'w') as mor_out:
        json.dump(morph_array, mor_out)
    with open(ner_file_name, 'w') as ner_out:
        json.dump(ner_parse.as_json, ner_out)
    with open(lem_file_name, 'w') as lem_out:
        json.dump(lemmatized, lem_out)


    return None

In [14]:
def create_and_save_parses(file_name):
    #загрузка парсеров
    navec = Navec.load('../parser_modules/navec_news_v1_1B_250K_300d_100q.tar')
    syntax = Syntax.load('../parser_modules/slovnet_syntax_news_v1.tar')
    morph = Morph.load('../parser_modules/slovnet_morph_news_v1.tar', batch_size=4)
    ner = NER.load('../parser_modules/slovnet_ner_news_v1.tar')

    syntax.navec(navec)
    morph.navec(navec)
    ner.navec(navec)
    
    syntax_parse, morph_parse, ner_parse, lemmatized = create_parses(file_name, parsers = [syntax, morph, ner])
    save_parses(file_name, syntax_parse, morph_parse, ner_parse, lemmatized)

    return None
    

In [15]:
papers = os.listdir("../struct_pub")

In [16]:
fname = "../struct_pub/" + papers[50] + "/main.json"

In [17]:
create_and_save_parses(fname)

In [18]:
#loading parses

def load_parses(file_name):
    syntx_file_name = file_name + '.' + 'syntx'
    morph_file_name = file_name + '.' + 'morph'
    ner_file_name = file_name + '.' + 'ner'
    lem_file_name = file_name + '.' + 'lem'

    syntax_parse = []
    morph_parse = []
    ner_parse = []

    with open(syntx_file_name, 'r') as syn_in:
        syntx_array = json.load(syn_in)
    with open(morph_file_name, 'r') as mor_in:
        morph_array = json.load(mor_in)
    with open(ner_file_name, 'r') as ner_in:
        ner_array = json.load(ner_in)
    with open(lem_file_name, 'r') as lem_in:
        lemmatized = json.load(lem_in)

    for sent in syntx_array:
        syntax_subarray = []
        for token in sent:
            syntax_subarray.append(slovnet.markup.SyntaxToken.from_json(token))
        syntax_parse.append(syntax_subarray)
        
    for sent in morph_array:
        morph_subarray = []
        for token in sent:
            morph_subarray.append(slovnet.markup.MorphToken.from_json(token))
        morph_parse.append(morph_subarray)
        
    ner_parse = slovnet.markup.SpanMarkup.from_json(ner_array)
    

    return syntax_parse, morph_parse, ner_parse, lemmatized


In [19]:
parses = load_parses(fname)

In [20]:
for sent in parses[0]:
    for token in sent:
        print(token)

SyntaxToken(id='1', text='9', head_id='1', rel='nummod')
SyntaxToken(id='2', text='1', head_id='1', rel='appos')
SyntaxToken(id='3', text='.', head_id='4', rel='punct')
SyntaxToken(id='4', text='Организация', head_id='4', rel='nsubj')
SyntaxToken(id='5', text='структур', head_id='9', rel='amod')
SyntaxToken(id='6', text='технических', head_id='9', rel='amod')
SyntaxToken(id='7', text='и', head_id='8', rel='cc')
SyntaxToken(id='8', text='программных', head_id='6', rel='conj')
SyntaxToken(id='9', text='средств', head_id='10', rel='conj')
SyntaxToken(id='10', text='проектирования', head_id='9', rel='nmod')
SyntaxToken(id='11', text='и', head_id='12', rel='cc')
SyntaxToken(id='12', text='управления', head_id='10', rel='conj')
SyntaxToken(id='13', text='.', head_id='1', rel='punct')
SyntaxToken(id='1', text='Средства', head_id='0', rel='root')
SyntaxToken(id='2', text='взаимодействия', head_id='1', rel='nmod')
SyntaxToken(id='3', text=',', head_id='4', rel='punct')
SyntaxToken(id='4', text=

In [61]:
for sent in parses[1]:
    for token in sent:
        print(token)

MorphToken(text='9', pos='NUM', feats={})
MorphToken(text='1', pos='NUM', feats={})
MorphToken(text='.', pos='PUNCT', feats={})
MorphToken(text='Организация', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'})
MorphToken(text='структур', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Fem', 'Number': 'Plur'})
MorphToken(text='технических', pos='ADJ', feats={'Case': 'Gen', 'Degree': 'Pos', 'Number': 'Plur'})
MorphToken(text='и', pos='CCONJ', feats={})
MorphToken(text='программных', pos='ADJ', feats={'Case': 'Gen', 'Degree': 'Pos', 'Number': 'Plur'})
MorphToken(text='средств', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Neut', 'Number': 'Plur'})
MorphToken(text='проектирования', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Neut', 'Number': 'Sing'})
MorphToken(text='и', pos='CCONJ', feats={})
MorphToken(text='управления', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Neut', 'Numb